In [ ]:
# Import libraries and set desired options
%matplotlib inline
from matplotlib import pyplot as plt

import pickle
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

Notebook by Yuri Kashnitsky, edited by Ivan Komarov. 

In this competition we are going to analyze a sequence of websites visited by a person to predict whether this person is Alice or not. The metric of evaluation is [ROC AUC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic). 

###  Data Downloading and Transformation
First, read the training and test sets. 

In [ ]:
times = ['time'+str(i) for i in range(1,11)]
times

['time1',
 'time2',
 'time3',
 'time4',
 'time5',
 'time6',
 'time7',
 'time8',
 'time9',
 'time10']

In [ ]:
!wget -O train.csv https://vk.com/doc161475809_632303117?hash=RXMmAO7vrz2RZBw448YBY9cZD9L8MozTxMQnHB6U4cc&dl=SkwAole7t9ubLpZhmjQGZ78baJKixrfoYfSISmcgwzw
!wget -O test.csv https://vk.com/doc161475809_632303101?hash=qnmPwIZHrMMcdJfSoz4Tzj1qyi3440AEIuC5suN5r4L&dl=7qWx76xmawFy8KXb4RU5DSfnTITjD5bn2FU15mEVJmH
!wget -O site_dic.pkl https://storage.yandexcloud.net/ds-ods/files/materials/a16eb5f3/site_dic.pkl

--2022-05-09 15:14:45--  https://vk.com/doc161475809_632303117?hash=RXMmAO7vrz2RZBw448YBY9cZD9L8MozTxMQnHB6U4cc
Resolving vk.com (vk.com)... 87.240.139.194, 87.240.137.158, 87.240.190.67, ...
Connecting to vk.com (vk.com)|87.240.139.194|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://psv4.userapi.com/c505536/u161475809/docs/d50/bdb12c037461/train_alice.csv?extra=AOP4pg0l19HxUEpo7hjrPOls5EH_VK_z3hmBdBrKO9R4HkyI0No1VHAER0IpM_S6Vfp13ynKCXiqE57s_Tf5AEjM6HEsCbJOwCv9TqI8Ju-JWF6z7BVyIC1bzT7tmpdSCgMfrDq0Vq8_Rw&dl=1 [following]
--2022-05-09 15:14:46--  https://psv4.userapi.com/c505536/u161475809/docs/d50/bdb12c037461/train_alice.csv?extra=AOP4pg0l19HxUEpo7hjrPOls5EH_VK_z3hmBdBrKO9R4HkyI0No1VHAER0IpM_S6Vfp13ynKCXiqE57s_Tf5AEjM6HEsCbJOwCv9TqI8Ju-JWF6z7BVyIC1bzT7tmpdSCgMfrDq0Vq8_Rw&dl=1
Resolving psv4.userapi.com (psv4.userapi.com)... 87.240.137.134, 87.240.137.140, 87.240.190.76, ...
Connecting to psv4.userapi.com (psv4.userapi.com)|87.240.137.134|:443... co

In [ ]:
# Read the training and test data sets and parse dates
train_df = pd.read_csv('train.csv',
                       index_col='session_id', parse_dates=times)

test_df = pd.read_csv('test.csv',
                      index_col='session_id', parse_dates=['time1'])

# Sort the data by time
train_df = train_df.sort_values(by='time1')

# Look at the first rows of the training set
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
27554,41475,2013-11-15 07:39:35,6725.0,2013-11-15 07:39:35,6725.0,2013-11-15 07:39:36,41475.0,2013-11-15 07:39:36,41476.0,2013-11-15 07:39:40,...,2013-11-15 07:39:41,6725.0,2013-11-15 07:42:50,41475.0,2013-11-15 07:42:50,41476.0,2013-11-15 07:42:50,6725.0,2013-11-15 07:44:25,0
81350,41476,2013-11-15 07:44:25,41475.0,2013-11-15 07:44:25,41476.0,2013-11-15 07:57:45,6725.0,2013-11-15 07:57:45,41475.0,2013-11-15 07:57:45,...,2013-11-15 07:57:46,41476.0,2013-11-15 07:57:47,6725.0,2013-11-15 07:57:49,41475.0,2013-11-15 07:57:49,41476.0,2013-11-15 07:57:49,0
234665,4802,2013-11-15 07:52:17,23.0,2013-11-15 07:52:18,4803.0,2013-11-15 07:52:19,38.0,2013-11-15 07:52:19,38.0,2013-11-15 07:52:20,...,2013-11-15 07:52:20,4804.0,2013-11-15 07:52:23,21.0,2013-11-15 07:52:26,23.0,2013-11-15 07:52:26,22.0,2013-11-15 07:52:28,0
97610,23,2013-11-15 07:52:28,23.0,2013-11-15 07:52:29,22.0,2013-11-15 07:52:37,21.0,2013-11-15 07:52:37,63.0,2013-11-15 07:55:10,...,2013-11-15 07:55:10,784.0,2013-11-15 07:55:56,4804.0,2013-11-15 07:57:50,4804.0,2013-11-15 08:01:18,784.0,2013-11-15 08:01:26,0
161358,41476,2013-11-15 07:57:50,41476.0,2013-11-15 07:57:51,6725.0,2013-11-15 07:59:34,41475.0,2013-11-15 07:59:34,41476.0,2013-11-15 07:59:34,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 27554 to 11690
Data columns (total 21 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   site1   253561 non-null  int64         
 1   time1   253561 non-null  datetime64[ns]
 2   site2   250098 non-null  float64       
 3   time2   250098 non-null  datetime64[ns]
 4   site3   246919 non-null  float64       
 5   time3   246919 non-null  datetime64[ns]
 6   site4   244321 non-null  float64       
 7   time4   244321 non-null  datetime64[ns]
 8   site5   241829 non-null  float64       
 9   time5   241829 non-null  datetime64[ns]
 10  site6   239495 non-null  float64       
 11  time6   239495 non-null  datetime64[ns]
 12  site7   237297 non-null  float64       
 13  time7   237297 non-null  datetime64[ns]
 14  site8   235224 non-null  float64       
 15  time8   235224 non-null  datetime64[ns]
 16  site9   233084 non-null  float64       
 17  time9   233084 non-null  d

**Watching class imbalance**

In [ ]:
a = train_df['target'].value_counts()
print(a)

0    251264
1      2297
Name: target, dtype: int64


percentage of Alice's sessions out of all sessions

In [ ]:
a[1] / (a[0] + a[1])

0.00905896411514389

First appearance Alice

In [ ]:
first_appearance = train_df[train_df['target'] == 1]['time1'].iloc[0]
first_appearance.isoweekday()

5

Years in train_df

In [ ]:
pd.DatetimeIndex(train_df['time1']).year.value_counts()

2014    178886
2013     74675
Name: time1, dtype: int64

Most visited site1 by Alice and the average user

In [ ]:
most_visited_alice = train_df[train_df['target'] == 1]['site1'].value_counts().index[0]
print(most_visited_alice)

most_visited_user = train_df[train_df['target'] == 0]['site1'].value_counts().index[0]
print(most_visited_user)


80
21


In [ ]:
with open(r"site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

In [ ]:
for k, v in site_dict.items():
    if v == most_visited_alice or v == most_visited_user:
        print(v, k)

80 s.youtube.com
21 www.google.fr


The training data set contains the following features:

- **site1** – ID of the first visited website in the session
- **time1** – visiting time for the first website in the session
- ...
- **site10** – ID of the tenth visited website in the session
- **time10** – visiting time for the tenth website in the session
- **target** – target variable, equals 1 for Alice's sessions, and 0 otherwise
    
**User sessions end either if a user has visited ten websites or if a session has lasted over thirty minutes.**

There are some empty values in the table, it means that some sessions contain less than ten websites. Replace empty values with 0 and change columns types to integer. Also load the websites dictionary and check how it looks:

In [ ]:
# Change site1, ..., site10 columns type to integer and fill NA-values with zeros
sites = ['site'+str(i) for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

# Load websites dictionary
with open(r"site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)
    
# r before a string means "raw", i.e. take the string as it comes,
# e.g. as a file path without interpreting special symbols like \n

print('Websites total:', len(site_dict))



Websites total: 48371


Какой сайт в обучающей выборке является вторым по популярности в 2014 году и сколько раз его посещали в 2014 году?

In [ ]:
def add_dict(dct: dict, key, value) -> None:
    if dct.get(key, None) is None:
        dct[key] = value
    else:
        dct[key] += value


In [ ]:
train_df_2014 = train_df[pd.DatetimeIndex(train_df['time1']).year == 2014]
site_numVisits = train_df_2014['site1'].value_counts().to_dict()

for site_feature in sites[1:]:
    tmp_site = train_df_2014[site_feature].value_counts().to_dict()
    for site, num_visit in tmp_site.items():
        add_dict(site_numVisits, site, num_visit)

In [ ]:
sorted_tuples = sorted(site_numVisits.items(), key=lambda item: item[1], reverse=True)
site_numVisits = {k: v for k, v in sorted_tuples}

# первые 5 значений
for i, (k, v) in enumerate(site_numVisits.items()):
    print(k, v)
    if i == 5:
        break

21 86090
0 81887
782 77039
23 56333
780 40610
22 39681


In [ ]:
top2_views_id_site = 782
for k, v in site_dict.items():
    if v == top2_views_id_site:
        print(v, k)

782 annotathon.org


Сколько всего сессий обучающей выборке содержат в себе меньше чем 10 сайтов?

In [ ]:
sites_sessions = (train_df[sites] != 0).sum(axis=1).value_counts()
sites_sessions

10    231052
1       3463
2       3179
3       2598
4       2492
5       2334
6       2198
8       2140
7       2073
9       2032
dtype: int64

In [ ]:
sum(sites_sessions.to_list()[1:])

22509

Выберите верные утверждения для обучающей выборки:

- Элис наиболее активна в марте и неактивна летом
- Лето является самым малоактивным временем года вообще

In [ ]:
train_df_alice = train_df[train_df['target'] == 1]
pd.DatetimeIndex(train_df_alice['time1']).month.value_counts()

12    610
2     449
11    446
3     399
4     305
1      88
Name: time1, dtype: int64

- В среднем Элис проводит меньше времени на первой странице, чем другие пользователи

In [164]:
first_duration_alice = (train_df_alice['time2'] - train_df_alice['time1']).apply(lambda x: x.total_seconds()).dropna().astype(int)
first_duration_alice.mean()

6.397994768962511

In [163]:
train_df_users = train_df[train_df['target'] == 0]
first_duration_users = (train_df_users['time2'] - train_df_users['time1']).apply(lambda x: x.total_seconds()).dropna().astype(int)
first_duration_users.mean()

19.317000532679053

- Медианная длительность посещения первой страницы в среднем больше, чем у Элис

In [165]:
print(first_duration_alice.median())
print(first_duration_users.median())

1.0
1.0


8) Чему равное медианное значение количества уникальных сайтов в рамках одной сессии обучающей выборки?



In [199]:
train_df_t = train_df[sites].T
count_unique_in_sessions = []
for column in train_df_t.columns:
    count_unique_in_sessions.append(len(train_df_t[column].unique()))

In [202]:
np.median(count_unique_in_sessions)

6.0

9) Чему равное медианное значение времени сессии в обучающей выборке (время посещения последнего сайта примем равным нулю)?



In [211]:
time1_10 = train_df[['time1','time10']].dropna()
duration_session_in_sec = (time1_10['time10'] - time1_10['time1']).apply(lambda x: x.total_seconds()).to_list()

In [212]:
np.median(duration_session_in_sec)

27.0

10) В каком месяце доля сессий Элис среди всех наибольшая?

In [218]:
# сессии Элис по месяцам
alice_sessions_month = pd.DatetimeIndex(train_df_alice['time1']).month.value_counts().to_dict()
alice_sessions_month

{1: 88, 2: 449, 3: 399, 4: 305, 11: 446, 12: 610}

In [220]:
# сессии других пользователей по месяцам
user_sessions_month = pd.DatetimeIndex(train_df_users['time1']).month.value_counts().to_dict()
user_sessions_month

{1: 27970, 2: 52659, 3: 54876, 4: 42053, 5: 87, 11: 30672, 12: 42947}

In [223]:
for key in alice_sessions_month.keys():
    print(f"month: {key} | ratio: {alice_sessions_month[key] / user_sessions_month[key]}")

month: 12 | ratio: 0.014203553216755536
month: 2 | ratio: 0.008526557663457338
month: 11 | ratio: 0.014540949400104329
month: 3 | ratio: 0.007270938115022961
month: 4 | ratio: 0.007252752479014577
month: 1 | ratio: 0.0031462281015373615


In [ ]:
# See what's in the dict
list(site_dict.items())[:3]

[('www.abmecatronique.com', 25075),
 ('groups.live.com', 13997),
 ('majeureliguefootball.wordpress.com', 42436)]

In [ ]:
# Size of the sets
print(test_df.shape, train_df.shape)

(82797, 20) (253561, 21)


In [142]:
# What's inside the train
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
27554,41475,2013-11-15 07:39:35,6725,2013-11-15 07:39:35,6725,2013-11-15 07:39:36,41475,2013-11-15 07:39:36,41476,2013-11-15 07:39:40,...,2013-11-15 07:39:41,6725,2013-11-15 07:42:50,41475,2013-11-15 07:42:50,41476,2013-11-15 07:42:50,6725,2013-11-15 07:44:25,0
81350,41476,2013-11-15 07:44:25,41475,2013-11-15 07:44:25,41476,2013-11-15 07:57:45,6725,2013-11-15 07:57:45,41475,2013-11-15 07:57:45,...,2013-11-15 07:57:46,41476,2013-11-15 07:57:47,6725,2013-11-15 07:57:49,41475,2013-11-15 07:57:49,41476,2013-11-15 07:57:49,0
234665,4802,2013-11-15 07:52:17,23,2013-11-15 07:52:18,4803,2013-11-15 07:52:19,38,2013-11-15 07:52:19,38,2013-11-15 07:52:20,...,2013-11-15 07:52:20,4804,2013-11-15 07:52:23,21,2013-11-15 07:52:26,23,2013-11-15 07:52:26,22,2013-11-15 07:52:28,0
97610,23,2013-11-15 07:52:28,23,2013-11-15 07:52:29,22,2013-11-15 07:52:37,21,2013-11-15 07:52:37,63,2013-11-15 07:55:10,...,2013-11-15 07:55:10,784,2013-11-15 07:55:56,4804,2013-11-15 07:57:50,4804,2013-11-15 08:01:18,784,2013-11-15 08:01:26,0
161358,41476,2013-11-15 07:57:50,41476,2013-11-15 07:57:51,6725,2013-11-15 07:59:34,41475,2013-11-15 07:59:34,41476,2013-11-15 07:59:34,...,NaT,0,NaT,0,NaT,0,NaT,0,NaT,0


For the very basic model, we will use only the visited websites in the session (we will not take into account timestamp features). 

*Alice has her favorite sites, and the more often you see these sites in the session, the higher probability that this is an Alice session, and vice versa.*

Let us prepare the data, we will take only features `site1, site2, ... , site10` from the whole dataframe. Keep in mind that the missing values are replaced with zero. Here is what the first rows of the dataframe look like:

In [ ]:
train_df[sites].head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
27554,41475,6725,6725,41475,41476,41476,6725,41475,41476,6725
81350,41476,41475,41476,6725,41475,41476,41476,6725,41475,41476
234665,4802,23,4803,38,38,4804,4804,21,23,22
97610,23,23,22,21,63,66,784,4804,4804,784
161358,41476,41476,6725,41475,41476,0,0,0,0,0


Since IDs of sites have no meaning (does not matter if a site has an ID of 1 or 100), we need to think about how to encode the meaning of "this site in a session means higher probablity that it is an Alice session". 

We will use a technique called ["bag of words plus n-gram model"](https://en.wikipedia.org/wiki/Bag-of-words_model).

We will make a "site-session" matrix analogous to the term-document matrix.

We are not the first, and luckily there is a function CountVectorizer that will implement the above model. Type help(CountVectorizer) to learn about the function. 

We will now initialize a "cv" (CountVectorizer's) instance which we need to train. 

We will use the following parameters:

_ngram range=(1, 3)_ - here we decide that we will use 
1) the name of the site, 
2) two consecutive site names, and 
3) three consecutive site names as features. 
E.g. "google.com" or "google.com vk.com" or "google.com vk.com groups.live.com". 

CountVectorizer will create a large dictionary of 1, 2, and 3-gram strings of sites represented by their numerical IDs. However, this dictionary will be so so large that we may run into trouble with memory or we will just be inefficent chasing phantom combinations.

We will thus limit the dictionary to 50K of the most frequent n-grams:

_max features=50000_

Here is our empty instance:

In [ ]:
cv = CountVectorizer(ngram_range=(1, 3), max_features=50000)

CountVectorizer accepts "document strings", so let's prepare a string of our "documents" (i.e. sites), divided by space. Since the string will be huge, we will write this string in a text file using pandas:

In [ ]:
train_df[sites].fillna(0).to_csv('train_sessions_text.txt', 
                                 sep=' ', index=None, header=None)
test_df[sites].fillna(0).to_csv('test_sessions_text.txt', 
                                sep=' ', index=None, header=None)

Before we start using CountVectorizer, let's see how it works on a sub-set of 5 sessions:

In [ ]:
five_sess = pd.read_csv('train_sessions_text.txt', sep=' ', nrows=5, header=None)

In [ ]:
five_sess

,0,1,2,3,4,5,6,7,8,9
0,41475,6725,6725,41475,41476,41476,6725,41475,41476,6725
1,41476,41475,41476,6725,41475,41476,41476,6725,41475,41476
2,4802,23,4803,38,38,4804,4804,21,23,22
3,23,23,22,21,63,66,784,4804,4804,784
4,41476,41476,6725,41475,41476,0,0,0,0,0


First of all, let's make an inverse dictionary which gives us a site name for ID.
The direct dictionary came to us like this:

In [ ]:
list(site_dict.items())[:3]

[('www.abmecatronique.com', 25075),
 ('groups.live.com', 13997),
 ('majeureliguefootball.wordpress.com', 42436)]

In [ ]:
# The inverse dictionary:

new_dict = {}
for key in site_dict:
    new_dict[site_dict[key]] = key

In [ ]:
# Let's check what's in it:

list(new_dict.items())[:3]

[(25075, 'www.abmecatronique.com'),
 (13997, 'groups.live.com'),
 (42436, 'majeureliguefootball.wordpress.com')]

In [ ]:
# Let's see site names in the five first sessions:

list_sites = []
for row in five_sess.values:
    row_sites = ' '.join([str(i) for i in row if i!=0])
    print(row_sites)
    list_sites.append(row_sites) 

print()
    
list_sites_names = []
for row in five_sess.values:
    row_sites = ' '.join([new_dict[i] for i in row if i!=0])
    print(row_sites)
    list_sites_names.append(row_sites)

41475 6725 6725 41475 41476 41476 6725 41475 41476 6725
41476 41475 41476 6725 41475 41476 41476 6725 41475 41476
4802 23 4803 38 38 4804 4804 21 23 22
23 23 22 21 63 66 784 4804 4804 784
41476 41476 6725 41475 41476

security.debian.org www-fourier.ujf-grenoble.fr www-fourier.ujf-grenoble.fr security.debian.org backports.debian.org backports.debian.org www-fourier.ujf-grenoble.fr security.debian.org backports.debian.org www-fourier.ujf-grenoble.fr
backports.debian.org security.debian.org backports.debian.org www-fourier.ujf-grenoble.fr security.debian.org backports.debian.org backports.debian.org www-fourier.ujf-grenoble.fr security.debian.org backports.debian.org
cnfg.toolbarservices.com www.google.com utils.delta-search.com ajax.googleapis.com ajax.googleapis.com img.babylon.com img.babylon.com www.google.fr www.google.com apis.google.com
www.google.com www.google.com apis.google.com www.google.fr ieonline.microsoft.com go.microsoft.com javadl-esd-secure.oracle.com img.babylon.com i

Here is what the fit and transform method -- i.e. learn the dictionary and make the matrix -- produces in our "cv":
a sparse matrix. Why sparse? Because nrows * dict_size = usually will not fit in memory 
(obviously, our 5 sessions will fit in memory so that we can look at them)

In [ ]:
see_vect = cv.fit_transform(list_sites)

# Matrix dimensions: 5 sessions of 60 elements
see_vect

<5x60 sparse matrix of type '<class 'numpy.int64'>'
	with 88 stored elements in Compressed Sparse Row format>

In [ ]:
# Here is the dictionary of sites, 1 to 3-gram words. First 6 elements in the matrix:

cv.get_feature_names()[:6]

['21', '21 23', '21 23 22', '21 63', '21 63 66', '22']

In [ ]:
# A version with the site names. Note that security.debian.org has ID of 21.

for i, string in enumerate(cv.get_feature_names()):
    if i < 21:
        print (i+1, end=" ")
        for num in string.split():
            print(new_dict[int(num)], end=" ")
        print()

1 www.google.fr 
2 www.google.fr www.google.com 
3 www.google.fr www.google.com apis.google.com 
4 www.google.fr ieonline.microsoft.com 
5 www.google.fr ieonline.microsoft.com go.microsoft.com 
6 apis.google.com 
7 apis.google.com www.google.fr 
8 apis.google.com www.google.fr ieonline.microsoft.com 
9 www.google.com 
10 www.google.com apis.google.com 
11 www.google.com apis.google.com www.google.fr 
12 www.google.com www.google.com 
13 www.google.com www.google.com apis.google.com 
14 www.google.com utils.delta-search.com 
15 www.google.com utils.delta-search.com ajax.googleapis.com 
16 ajax.googleapis.com 
17 ajax.googleapis.com ajax.googleapis.com 
18 ajax.googleapis.com ajax.googleapis.com img.babylon.com 
19 ajax.googleapis.com img.babylon.com 
20 ajax.googleapis.com img.babylon.com img.babylon.com 
21 security.debian.org 


In [ ]:
# Here is the session-site matrix, toarrray() helps us to see a sparse matrix since it is not large.

see_vect.toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 2,
        1, 1, 1, 1, 3, 0, 0, 1, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 4, 2, 2, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3,
        1, 1, 0, 0, 5, 1, 1, 1, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 1, 1, 2, 1, 1, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 2, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        0, 0, 0, 0, 3, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
# The first session (row in the matrix) is this:

list_sites_names[0]

'security.debian.org www-fourier.ujf-grenoble.fr www-fourier.ujf-grenoble.fr security.debian.org backports.debian.org backports.debian.org www-fourier.ujf-grenoble.fr security.debian.org backports.debian.org www-fourier.ujf-grenoble.fr'

Let's see how the first site of the first session, "security.debian.org", is recorded in the session-site matrix. 
Its ID is 21 which corresponds to 3. It is the number of times this site was seen in the first session.
Indeed, count for yourself in the cell above. 

In [ ]:

first_row = see_vect.toarray()[0]

for one, two in zip(range(60),first_row):
    if one < 21:
        print (one+1, two)


1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0
10 0
11 0
12 0
13 0
14 0
15 0
16 0
17 0
18 0
19 0
20 0
21 3


Let's go back to all sessions.

Fit `CountVectorizer` to train data and transform the train and test data with it.

In [ ]:
%%time

with open('train_sessions_text.txt') as inp_train_file:
    X_train = cv.fit_transform(inp_train_file)
with open('test_sessions_text.txt') as inp_test_file:
    X_test = cv.transform(inp_test_file)

print(X_train.shape, X_test.shape)

# Note very big dimensions of matrices: 253561 * 50000 = 12678050000 elements in train! Only sparse matrices can take it.

(253561, 50000) (82797, 50000)
Wall time: 21.2 s


NameError: ignored

### Training the first model

So, we have an algorithm and data for it. Let us build our first model, using [logistic regression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) implementation from ` Sklearn` with default parameters. We will use the first 90% of the data for training (the training data set is sorted by time) and the remaining 10% for validation. Let's write a simple function that returns the quality of the model and then train our first classifier:

In [ ]:
def get_auc_lr_valid(X, y, C=1.0, seed=17, ratio = 0.9):
    
    # Split the data into the training and validation sets
    idx = int(round(X.shape[0] * ratio))
    
    # Classifier training
    lr = LogisticRegression(C=C, random_state=seed, solver='lbfgs', max_iter=500).fit(X[:idx, :], y[:idx])
    
    # Prediction for validation set
    y_pred = lr.predict_proba(X[idx:, :])[:, 1]
    
    # Calculate the quality
    score = roc_auc_score(y[idx:], y_pred)
    
    return score

In [ ]:
# Our target variable
y_train = train_df['target'].values

In [ ]:
%%time
# Calculate metric on the validation set. 90% of train data for training. 10% for validation.

print(get_auc_lr_valid(X_train, y_train))

0.9122581928805027
Wall time: 21 s


In [ ]:
# 50% of train data for training:

get_auc_lr_valid(X_train, y_train, ratio=0.5)

0.8225182301089249

In [ ]:
# Wow! Big data rules in this task: .82 -> .91

In [ ]:
# Function for writing predictions to a file
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = range(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [ ]:
# Train the model on the whole training data set
# Use random_state=17 for reproducibility
# Parameter C=1 by default, but here we set it explicitly

lr = LogisticRegression(C=1.0, random_state=17, solver='lbfgs', max_iter=500).fit(X_train, y_train)

# Make a prediction for test data set
y_test = lr.predict_proba(X_test)[:, 1]

# Write it to the file which could be submitted
write_to_submission_file(y_test, 'baseline_1.csv')

The first model demonstrated the quality of 0.9235 on the validation set. Let's take it as the first baseline and starting point. 